Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [27]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + 
      0.01*tf.nn.l2_loss(weights) + 0.01*tf.nn.l2_loss(biases))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 47.699238
Minibatch accuracy: 10.2%
Validation accuracy: 15.3%
Minibatch loss at step 500: 0.871051
Minibatch accuracy: 81.2%
Validation accuracy: 80.4%
Minibatch loss at step 1000: 0.640312
Minibatch accuracy: 82.8%
Validation accuracy: 80.7%
Minibatch loss at step 1500: 0.772006
Minibatch accuracy: 83.6%
Validation accuracy: 81.0%
Minibatch loss at step 2000: 0.760200
Minibatch accuracy: 78.9%
Validation accuracy: 81.0%
Minibatch loss at step 2500: 0.726092
Minibatch accuracy: 85.9%
Validation accuracy: 80.3%
Minibatch loss at step 3000: 0.728443
Minibatch accuracy: 82.0%
Validation accuracy: 81.4%
Test accuracy: 88.2%


In [7]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases = tf.Variable(tf.zeros([hidden_nodes]))
    
  weights1 = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases1 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation with hidden layer
  hidden_layer_output = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)
  logits = tf.matmul(hidden_layer_output, weights1) + biases1
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)+ 
      0.01*tf.nn.l2_loss(weights) + 0.01*tf.nn.l2_loss(biases) +
      0.01*tf.nn.l2_loss(weights1) + 0.01*tf.nn.l2_loss(biases1))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden, weights1) + biases1)
  test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(test_hidden, weights1) + biases1)

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3489.659668
Minibatch accuracy: 7.8%
Validation accuracy: 35.8%
Minibatch loss at step 500: 21.270103
Minibatch accuracy: 84.4%
Validation accuracy: 84.3%
Minibatch loss at step 1000: 0.783298
Minibatch accuracy: 85.2%
Validation accuracy: 83.7%
Minibatch loss at step 1500: 0.730899
Minibatch accuracy: 85.2%
Validation accuracy: 82.7%
Minibatch loss at step 2000: 0.725304
Minibatch accuracy: 84.4%
Validation accuracy: 83.8%
Minibatch loss at step 2500: 0.711898
Minibatch accuracy: 85.2%
Validation accuracy: 83.4%
Minibatch loss at step 3000: 0.682801
Minibatch accuracy: 84.4%
Validation accuracy: 83.7%
Test accuracy: 90.2%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [5]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases = tf.Variable(tf.zeros([hidden_nodes]))
    
  weights1 = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases1 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation with hidden layer
  hidden_layer_output = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)
  logits = tf.matmul(hidden_layer_output, weights1) + biases1
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden, weights1) + biases1)
  test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(test_hidden, weights1) + biases1)

In [13]:
num_steps = 31
num_batches = 3

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = step % num_batches
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 2 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 480.298126
Minibatch accuracy: 11.7%
Validation accuracy: 28.5%
Minibatch loss at step 2: 1137.733521
Minibatch accuracy: 36.7%
Validation accuracy: 48.9%
Minibatch loss at step 4: 144.013641
Minibatch accuracy: 71.1%
Validation accuracy: 65.9%
Minibatch loss at step 6: 8.639128
Minibatch accuracy: 93.8%
Validation accuracy: 69.1%
Minibatch loss at step 8: 8.708739
Minibatch accuracy: 96.1%
Validation accuracy: 68.3%
Minibatch loss at step 10: 0.205051
Minibatch accuracy: 99.2%
Validation accuracy: 68.7%
Minibatch loss at step 12: 5.414773
Minibatch accuracy: 96.1%
Validation accuracy: 68.6%
Minibatch loss at step 14: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 69.0%
Minibatch loss at step 16: 0.164951
Minibatch accuracy: 99.2%
Validation accuracy: 68.7%
Minibatch loss at step 18: 4.416961
Minibatch accuracy: 98.4%
Validation accuracy: 68.6%
Minibatch loss at step 20: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 68.2%
Minib

The accuracy of the training set is high and accuracies for validation and test accuracy are low which is happening because if overfitting.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [28]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases = tf.Variable(tf.zeros([hidden_nodes]))
    
  weights1 = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases1 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation with hidden layer
  hidden_layer_output = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)
  drop_out = tf.nn.dropout(hidden_layer_output, 0.5)
  logits = tf.matmul(drop_out, weights1) + biases1
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden, weights1) + biases1)
  test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(test_hidden, weights1) + biases1)

In [29]:
num_steps = 31
num_batches = 3

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = step % num_batches
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 2 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 509.293579
Minibatch accuracy: 10.2%
Validation accuracy: 46.7%
Minibatch loss at step 2: 370.075745
Minibatch accuracy: 54.7%
Validation accuracy: 48.5%
Minibatch loss at step 4: 260.057556
Minibatch accuracy: 69.5%
Validation accuracy: 57.7%
Minibatch loss at step 6: 61.841400
Minibatch accuracy: 86.7%
Validation accuracy: 68.1%
Minibatch loss at step 8: 16.639769
Minibatch accuracy: 95.3%
Validation accuracy: 67.6%
Minibatch loss at step 10: 7.174233
Minibatch accuracy: 96.1%
Validation accuracy: 69.1%
Minibatch loss at step 12: 3.581245
Minibatch accuracy: 96.9%
Validation accuracy: 68.2%
Minibatch loss at step 14: 1.533498
Minibatch accuracy: 98.4%
Validation accuracy: 68.2%
Minibatch loss at step 16: 2.861639
Minibatch accuracy: 97.7%
Validation accuracy: 68.4%
Minibatch loss at step 18: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 68.0%
Minibatch loss at step 20: 3.880054
Minibatch accuracy: 98.4%
Validation accuracy: 69.3%
Minib

Dropout did not improve the performance.

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


Lets try with exponential decay first and one hidden layer.

In [73]:
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases = tf.Variable(tf.zeros([hidden_nodes]))
    
  weights1 = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases1 = tf.Variable(tf.zeros([num_labels]))
  global_step = tf.Variable(0, trainable=False)
  starter_learning_rate = 0.1
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           100000, 0.96, staircase=True)
  # Training computation with hidden layer
  hidden_layer_output = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)
  logits = tf.matmul(hidden_layer_output, weights1) + biases1
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)+ 
      0.01*tf.nn.l2_loss(weights) + 0.01*tf.nn.l2_loss(biases) +
      0.01*tf.nn.l2_loss(weights1) + 0.01*tf.nn.l2_loss(biases1))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden, weights1) + biases1)
  test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(test_hidden, weights1) + biases1)

In [74]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3647.746582
Minibatch accuracy: 6.2%
Validation accuracy: 23.1%
Minibatch loss at step 500: 1158.393433
Minibatch accuracy: 78.1%
Validation accuracy: 81.0%
Minibatch loss at step 1000: 421.938873
Minibatch accuracy: 88.3%
Validation accuracy: 81.8%
Minibatch loss at step 1500: 154.878723
Minibatch accuracy: 89.1%
Validation accuracy: 84.4%
Minibatch loss at step 2000: 57.257111
Minibatch accuracy: 82.0%
Validation accuracy: 86.0%
Minibatch loss at step 2500: 21.396931
Minibatch accuracy: 85.9%
Validation accuracy: 86.0%
Minibatch loss at step 3000: 8.245090
Minibatch accuracy: 86.7%
Validation accuracy: 85.6%
Test accuracy: 91.7%


Lets try with two hidden layers.

In [75]:
batch_size = 128
hidden_nodes1 = 1024
hidden_nodes2 = 100
regul = 1e-3

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes1], 
                        stddev=np.sqrt(2.0 / (image_size * image_size))))
  biases1 = tf.Variable(tf.zeros([hidden_nodes1]))
    
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_nodes1, hidden_nodes2], 
                        stddev=np.sqrt(2.0 / hidden_nodes1)))
  biases2 = tf.Variable(tf.zeros([hidden_nodes2]))
  global_step = tf.Variable(0, trainable=False)
  
  weights3 = tf.Variable(
    tf.truncated_normal([hidden_nodes2, num_labels],
                       stddev=np.sqrt(2.0 / hidden_nodes2)))
  biases3 = tf.Variable(tf.zeros([num_labels]))
  global_step = tf.Variable(0, trainable=False)

  starter_learning_rate = 0.1
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           100000, 0.96, staircase=True)
  # Training computation with hidden layer
  hidden_layer_output1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  hidden_layer_output2 = tf.nn.relu(tf.matmul(hidden_layer_output1, weights2) + biases2)
  logits = tf.matmul(hidden_layer_output2, weights3) + biases3
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)+
      regul*tf.nn.l2_loss(weights3) + regul*tf.nn.l2_loss(biases3) +
      regul*tf.nn.l2_loss(weights2) + regul*tf.nn.l2_loss(biases2) +
      regul*tf.nn.l2_loss(weights1) + regul*tf.nn.l2_loss(biases1))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_hidden1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  valid_hidden2 = tf.nn.relu(tf.matmul(valid_hidden1, weights2) + biases2)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden2, weights3) + biases3)
    
  test_hidden1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  test_hidden2 = tf.nn.relu(tf.matmul(test_hidden1, weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(test_hidden2, weights3) + biases3)

In [76]:
num_steps = 9001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.204624
Minibatch accuracy: 19.5%
Validation accuracy: 25.9%
Minibatch loss at step 500: 1.280182
Minibatch accuracy: 86.7%
Validation accuracy: 84.8%
Minibatch loss at step 1000: 1.077919
Minibatch accuracy: 88.3%
Validation accuracy: 86.4%
Minibatch loss at step 1500: 1.066951
Minibatch accuracy: 90.6%
Validation accuracy: 87.0%
Minibatch loss at step 2000: 1.093388
Minibatch accuracy: 83.6%
Validation accuracy: 87.4%
Minibatch loss at step 2500: 0.951535
Minibatch accuracy: 88.3%
Validation accuracy: 88.2%
Minibatch loss at step 3000: 0.833848
Minibatch accuracy: 91.4%
Validation accuracy: 88.5%
Minibatch loss at step 3500: 0.798614
Minibatch accuracy: 90.6%
Validation accuracy: 89.0%
Minibatch loss at step 4000: 0.919675
Minibatch accuracy: 84.4%
Validation accuracy: 89.1%
Minibatch loss at step 4500: 0.666000
Minibatch accuracy: 89.8%
Validation accuracy: 89.4%
Minibatch loss at step 5000: 0.715577
Minibatch accuracy: 87.5%
Validation accurac

Initialize the weights with a correct standard deviation.

In [77]:
batch_size = 128
hidden_nodes1 = 1024
hidden_nodes2 = 256
hidden_nodes3 = 128
regul = 1e-3

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes1], 
                        stddev=np.sqrt(2.0 / (image_size * image_size))))
  biases1 = tf.Variable(tf.zeros([hidden_nodes1]))
    
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_nodes1, hidden_nodes2], 
                        stddev=np.sqrt(2.0 / hidden_nodes1)))
  biases2 = tf.Variable(tf.zeros([hidden_nodes2]))
  global_step = tf.Variable(0, trainable=False)
  
  weights3 = tf.Variable(
    tf.truncated_normal([hidden_nodes2, hidden_nodes3],
                       stddev=np.sqrt(2.0 / hidden_nodes2)))
  biases3 = tf.Variable(tf.zeros([hidden_nodes3]))
  global_step = tf.Variable(0, trainable=False)

  weights4 = tf.Variable(
    tf.truncated_normal([hidden_nodes3, num_labels],
                       stddev=np.sqrt(2.0 / hidden_nodes3)))
  biases4 = tf.Variable(tf.zeros([num_labels]))
  global_step = tf.Variable(0, trainable=False)

  starter_learning_rate = 0.5
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           4000, 0.65, staircase=True)
  # Training computation with hidden layer
  hidden_layer_output1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  hidden_layer_output2 = tf.nn.relu(tf.matmul(hidden_layer_output1, weights2) + biases2)
  hidden_layer_output3 = tf.nn.relu(tf.matmul(hidden_layer_output2, weights3) + biases3)
  logits = tf.matmul(hidden_layer_output3, weights4) + biases4
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_hidden1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  valid_hidden2 = tf.nn.relu(tf.matmul(valid_hidden1, weights2) + biases2)
  valid_hidden3 = tf.nn.relu(tf.matmul(valid_hidden2, weights3) + biases3)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden3, weights4) + biases4)
    
  test_hidden1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  test_hidden2 = tf.nn.relu(tf.matmul(test_hidden1, weights2) + biases2)
  test_hidden3 = tf.nn.relu(tf.matmul(test_hidden2, weights3) + biases3)
  test_prediction = tf.nn.softmax(tf.matmul(test_hidden3, weights4) + biases4)

In [78]:
num_steps = 18001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.446685
Minibatch accuracy: 10.9%
Validation accuracy: 28.8%
Minibatch loss at step 500: 0.404650
Minibatch accuracy: 86.7%
Validation accuracy: 85.6%
Minibatch loss at step 1000: 0.339003
Minibatch accuracy: 89.8%
Validation accuracy: 88.1%
Minibatch loss at step 1500: 0.299273
Minibatch accuracy: 90.6%
Validation accuracy: 88.0%
Minibatch loss at step 2000: 0.385818
Minibatch accuracy: 86.7%
Validation accuracy: 88.6%
Minibatch loss at step 2500: 0.352756
Minibatch accuracy: 87.5%
Validation accuracy: 89.3%
Minibatch loss at step 3000: 0.233093
Minibatch accuracy: 93.8%
Validation accuracy: 89.5%
Minibatch loss at step 3500: 0.230655
Minibatch accuracy: 93.8%
Validation accuracy: 89.7%
Minibatch loss at step 4000: 0.356342
Minibatch accuracy: 87.5%
Validation accuracy: 90.0%
Minibatch loss at step 4500: 0.188524
Minibatch accuracy: 94.5%
Validation accuracy: 90.6%
Minibatch loss at step 5000: 0.192065
Minibatch accuracy: 93.8%
Validation accurac

Now trying 3 hidden layers with dropout to hidden layers in training.

In [81]:
batch_size = 128
hidden_nodes1 = 1024
hidden_nodes2 = 1024
hidden_nodes3 = 256
regul = 1e-3
prob = 0.5

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes1], 
                        stddev=np.sqrt(2.0 / (image_size * image_size))))
  biases1 = tf.Variable(tf.zeros([hidden_nodes1]))
    
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_nodes1, hidden_nodes2], 
                        stddev=np.sqrt(2.0 / hidden_nodes1)))
  biases2 = tf.Variable(tf.zeros([hidden_nodes2]))
  global_step = tf.Variable(0, trainable=False)
  
  weights3 = tf.Variable(
    tf.truncated_normal([hidden_nodes2, hidden_nodes3],
                       stddev=np.sqrt(2.0 / hidden_nodes2)))
  biases3 = tf.Variable(tf.zeros([hidden_nodes3]))
  global_step = tf.Variable(0, trainable=False)

  weights4 = tf.Variable(
    tf.truncated_normal([hidden_nodes3, num_labels],
                       stddev=np.sqrt(2.0 / hidden_nodes3)))
  biases4 = tf.Variable(tf.zeros([num_labels]))
  global_step = tf.Variable(0, trainable=False)

  starter_learning_rate = 0.5
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           5000, 0.8, staircase=True)
  # Training computation with hidden layer
  hidden_layer_output1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  drop1 = tf.nn.dropout(hidden_layer_output1, prob)
  hidden_layer_output2 = tf.nn.relu(tf.matmul(drop1, weights2) + biases2)
  drop2 = tf.nn.dropout(hidden_layer_output2, prob)
  hidden_layer_output3 = tf.nn.relu(tf.matmul(drop2, weights3) + biases3)
  drop3 = tf.nn.dropout(hidden_layer_output3, prob)
  logits = tf.matmul(drop3, weights4) + biases4
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_hidden1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  valid_hidden2 = tf.nn.relu(tf.matmul(valid_hidden1, weights2) + biases2)
  valid_hidden3 = tf.nn.relu(tf.matmul(valid_hidden2, weights3) + biases3)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden3, weights4) + biases4)
    
  test_hidden1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  test_hidden2 = tf.nn.relu(tf.matmul(test_hidden1, weights2) + biases2)
  test_hidden3 = tf.nn.relu(tf.matmul(test_hidden2, weights3) + biases3)
  test_prediction = tf.nn.softmax(tf.matmul(test_hidden3, weights4) + biases4)

In [82]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.729622
Minibatch accuracy: 8.6%
Validation accuracy: 26.2%
Minibatch loss at step 500: 0.585251
Minibatch accuracy: 84.4%
Validation accuracy: 83.7%
Minibatch loss at step 1000: 0.541814
Minibatch accuracy: 85.9%
Validation accuracy: 85.4%
Minibatch loss at step 1500: 0.455521
Minibatch accuracy: 85.9%
Validation accuracy: 86.0%
Minibatch loss at step 2000: 0.557416
Minibatch accuracy: 83.6%
Validation accuracy: 86.6%
Minibatch loss at step 2500: 0.508445
Minibatch accuracy: 84.4%
Validation accuracy: 86.7%
Minibatch loss at step 3000: 0.428435
Minibatch accuracy: 85.9%
Validation accuracy: 87.1%
Minibatch loss at step 3500: 0.555073
Minibatch accuracy: 86.7%
Validation accuracy: 87.9%
Minibatch loss at step 4000: 0.600720
Minibatch accuracy: 82.0%
Validation accuracy: 88.1%
Minibatch loss at step 4500: 0.343781
Minibatch accuracy: 89.1%
Validation accuracy: 88.4%
Minibatch loss at step 5000: 0.507831
Minibatch accuracy: 86.7%
Validation accuracy